Insert explanatory text here. Allow 10 minutes for completion.

In [1]:
load("preamble.sage")

Apply the resultant criterion to genus 6 curves and report the resulting point counts.

In [2]:
d = defaultdict(list)
for t in targets6:
    u = weil_poly_from_point_count(t, 6)
    for (f,_) in u.factor():
        if f.degree() == 2:
            g = u//f
            h1 = f.trace_polynomial()[0]
            h2 = g.trace_polynomial()[0]
            if h1.resultant(h2) %2 == 0:
                c = f[1]+3
                d[c].append(t[:4])
[(c, set(d[c])) for c in d]

[(5, {(5, 13, 14, 25), (6, 10, 9, 38)}),
 (3, {(5, 13, 14, 25), (6, 10, 9, 38)}),
 (1, {(6, 10, 9, 38)})]

Apply the resultant criterion to genus 7 curves and report the resulting point counts.

In [3]:
d = defaultdict(list)
for t in targets7:
    u = weil_poly_from_point_count(t, 7)
    for (f,_) in u.factor():
        if f.degree() == 2:
            g = u//f
            h1 = f.trace_polynomial()[0]
            h2 = g.trace_polynomial()[0]
            if h1.resultant(h2) %2 == 0:
                c = f[1]+3
                d[c].append(t[:4])
[(c, set(d[c])) for c in d]

[(3, {(6, 18, 12, 18)}), (5, {(6, 18, 12, 18)})]

Construct the function field of the unique elliptic curve $E$ with $\#E(\mathbb{F}_2) = 3$.

In [4]:
P.<x> = GF(2)[]
E = HyperellipticCurve(x^3,P(1))
E1 = magma(E)
F0 = E1.FunctionField().AlgorithmicFunctionField()
assert F0.NumberOfPlacesOfDegreeOneECF(1) == 3

Collect double covers of $E$ ramified at the two degree-3 places with the desired point counts.

In [5]:
l = []
places3 = [i for i in F0.Places(3)]
assert len(places3) == 2
z = F0.DivisorGroup().Identity() + places3[0] + places3[1]
for F in cyclic_covers_by_ramification(F0, 2, 2*z, delta=6):
    t = tuple(F.NumberOfPlacesOfDegreeOneECF(i) for i in range(1,8))
    if t in targets7:
        l.append(F)
len(l)

1

Collect double covers of $E$ ramified at a degree-6 place with the desired point counts.

In [6]:
places6 = [i for i in F0.Places(6)]
for i in places6:
    z = F0.DivisorGroup().Identity() + i
    for F in cyclic_covers_by_ramification(F0, 2, 2*z, delta=6):
        t = tuple(F.NumberOfPlacesOfDegreeOneECF(i) for i in range(1,8))
        if t in targets7:
            l.append(F)
len(l)

3

In [7]:
l = isomorphism_class_reps(l)
len(l)

2

Search for extensions of relative class number 1.

In [8]:
l2 = []
for F in l:
    l2 += match_weil_poly(F, 2)
len(l2)

1

In [20]:
for (F1, F2) in l2:
    t = tuple(F1.NumberOfPlacesOfDegreeOneECF(i) for i in range(1,8))
    print(t, F1.RationalExtensionRepresentation())

(6, 18, 12, 18, 6, 60, 174) Algebraic function field defined over Univariate rational function field over GF(2) by
$.1^4 + (x^6 + x^4 + x^3 + x^2 + 1)*$.1^2 + (x^6 + x^4 + x^3 + x^2)*$.1 + x^10 + x^9 + x^7 + x^6


Convert the covers into numerical data in preparation for writing out to the spreadsheet.

In [10]:
l3 = []
Q.<T> = QQ[]
for (F, F1) in l2:
    u1 = T.parent()(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
    v1 = T.parent()(magma_poly_list(F1.ZetaFunction().Numerator())).reverse()
    l3.append((2, 7, 13, point_count_from_weil_poly(u1, 13), point_count_from_weil_poly(v1, 13)))

Write the results back to the spreadsheet. In the process, we check that every covering we found gives a pair of Weil polynomials from our original list.

In [11]:
used_pols = []
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    ct1 = eval(r["Counts of C"])
    ct2 = eval(r["Counts of C'"])
    if (d, g, g1, ct1, ct2) in l3:
        df.loc[i, "Cyclic"] = "Yes"
        used_pols.append((d, g, g1, ct1, ct2))
assert all(t in used_pols for t in l3)

In [12]:
df.to_excel('../Shared/polys.xlsx', sheet_name='Weil polynomials', merge_cells=True, freeze_panes=(int(1),int(1)))

In [13]:
print("This stage complete!")

This stage complete!
